In [3]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import time

In [4]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
elif socket.gethostname()=='LAPTOP-PUSGG08B11': #Ip del working
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656.dsv'

In [5]:
vej = pd.read_csv( ruta_vj, delimiter='\t', encoding='utf-8', decimal='.' )
vej['FECHA_NACIMIENTO'] = pd.to_datetime(vej['FECHA_NACIMIENTO'], format='%d/%m/%Y')
vej['FECHA_MUERTE'] = pd.to_datetime(vej['FECHA_MUERTE'], format='%d/%m/%Y')
vej['FECHA_DERECHO'] = pd.to_datetime(vej['FECHA_DERECHO'], format='%d/%m/%Y')
vej['RANGO_INI_5MEJ'] = pd.to_datetime(vej['RANGO_INI_5MEJ'], format='%d/%m/%Y')
vej['RANGO_FIN_5MEJ'] = pd.to_datetime(vej['RANGO_FIN_5MEJ'], format='%d/%m/%Y')

In [6]:
#Datos de vejez
vej.sort_values(by='CEDULA_COD',inplace=True)

In [7]:
vej[vej['CEDULA_COD']==13125042]

,CEDULA_COD,SEXO,FECHA_NACIMIENTO,FECHA_MUERTE,FECHA_DERECHO,NUM_IMPOSICIONES,COEFICIENTE_TEO,COEFICIENTE_REAL,BASE_AP_TEO,BASE_AP_REAL,PENSION,TIPO_PRESTACION,RANGO_INI_5MEJ,RANGO_FIN_5MEJ


In [8]:
#Se importa el archivo
afi = pd.read_csv(ruta_afi, delimiter='\t', encoding='utf-8', decimal='.')
#afi.sort_values(by='CEDULA_COD',inplace=True)
#afi=afi.set_index(['CEDULA_COD'])
# afi[afi['CEDULA_COD']==114]

In [9]:
afi

,CEDULA_COD,ANIO,MES,SALARIO,PORCENTAJE_AP,SECTOR
0,20057738,2003,3,135.03,20.5,PRIVADO
1,15613923,2003,11,65.61,20.5,PRIVADO
2,16057398,1996,11,56.65,20.5,PRIVADO
3,16023482,1997,9,15.11,20.5,PRIVADO
4,16021285,1998,1,27.27,20.5,PUBLICO
...,...,...,...,...,...,...
66555199,16773012,2005,9,230.00,20.5,PRIVADO
66555200,5709849,2007,1,278.95,30.5,PUBLICO
66555201,19960695,2008,9,1400.00,20.5,PRIVADO
66555202,11118795,1993,3,56.65,20.5,PRIVADO


In [10]:
afi['SECTOR'].unique()

array(['PRIVADO', 'PUBLICO', 'INDEPENDIENTES',
       '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  ',
       '69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
       '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA',
       '89-AFILIACION DOMICILIADO EN EL EXTERIOR', 'PASANTE PRIVADO',
       'PASANTE PUBLICO', '108-ARTISTA Y GESTOR DE CULTURA',
       '06-CODIGO DEL TRABAJO - CT '], dtype=object)

In [11]:
#Cedulas y aparición en cada sector
ced_sec = afi.groupby('CEDULA_COD')['SECTOR'].nunique()

In [12]:
ced_sec[ced_sec==2]

CEDULA_COD
216         2
288         2
387         2
397         2
402         2
           ..
20942932    2
20949683    2
21035907    2
21579009    2
23075971    2
Name: SECTOR, Length: 136187, dtype: int64

In [13]:
afi[afi['CEDULA_COD']==13125042]['SECTOR'].unique()

array(['PASANTE PRIVADO'], dtype=object)

In [14]:
#Cédulas que tiene un único sector
uni_ced_sec = ced_sec[ced_sec == 1].index.tolist()
#Cédulas que tiene un dos sector
dos_ced_sec = ced_sec[ced_sec == 2].index.tolist()
#Cédulas que tiene un tres sector
tres_ced_sec = ced_sec[ced_sec == 3].index.tolist()
#Cédulas que tiene más tres sector
mas_ced_sec = ced_sec[ced_sec > 3].index.tolist()

In [15]:
#Afliados de HL con un único sector
afi_uni = afi[afi['CEDULA_COD'].isin(uni_ced_sec)]
#Afliados de HL con un único sector
afi_dos = afi[afi['CEDULA_COD'].isin(dos_ced_sec)]
#Afliados de HL con un único sector
afi_tres = afi[afi['CEDULA_COD'].isin(tres_ced_sec)]
#Afliados de HL con un único sector
afi_mas = afi[afi['CEDULA_COD'].isin(mas_ced_sec)]

In [16]:
#Se presenta la clasificación
print('Se tiene un total ',afi['CEDULA_COD'].nunique(), ' personas que se va analizar su HL')
print('Se tiene un total ',afi_uni['CEDULA_COD'].nunique(), ' personas que tienen un único sector')
print('Se tiene un total ',afi_dos['CEDULA_COD'].nunique(), ' personas que tienen dos sectores')
print('Se tiene un total ',afi_tres['CEDULA_COD'].nunique(), ' personas que tienen tres sectores')
print('Se tiene un total ',afi_mas['CEDULA_COD'].nunique(), ' personas que tienen más de tres sectores')

Se tiene un total  556782  personas que se va analizar su HL
Se tiene un total  391458  personas que tienen un único sector
Se tiene un total  136187  personas que tienen dos sectores
Se tiene un total  26115  personas que tienen tres sectores
Se tiene un total  3022  personas que tienen más de tres sectores


In [17]:
391458+136187+26115+3022

556782

In [18]:
#Primero se va a trabajar con las cedúlas que tiene un único sector
afi_uni['SECTOR'].unique()

array(['PRIVADO', 'PUBLICO', 'INDEPENDIENTES',
       '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  ',
       '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA',
       '89-AFILIACION DOMICILIADO EN EL EXTERIOR',
       '108-ARTISTA Y GESTOR DE CULTURA', 'PASANTE PRIVADO',
       '69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
       'PASANTE PUBLICO'], dtype=object)

In [19]:
afi_uni['SECTOR'].unique()

array(['PRIVADO', 'PUBLICO', 'INDEPENDIENTES',
       '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  ',
       '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA',
       '89-AFILIACION DOMICILIADO EN EL EXTERIOR',
       '108-ARTISTA Y GESTOR DE CULTURA', 'PASANTE PRIVADO',
       '69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
       'PASANTE PUBLICO'], dtype=object)

In [20]:
#Se desagrea por sector 
pri = afi_uni[ (afi_uni['SECTOR']=='PRIVADO') | (afi_uni['SECTOR']=='PASANTE PRIVADO')]
pri = pri.sort_values( by=["CEDULA_COD", "ANIO", "MES"], ascending=[True, False, False] )

ind = afi_uni[ (afi_uni['SECTOR']=='INDEPENDIENTES') | (afi_uni['SECTOR']=='108-ARTISTA Y GESTOR DE CULTURA') |
               (afi_uni['SECTOR']=='02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  ')]
ind = ind.sort_values( by=["CEDULA_COD", "ANIO", "MES"], ascending=[True, False, False] )

pub = afi_uni[ (afi_uni['SECTOR']=='PUBLICO') | (afi_uni['SECTOR']=='PASANTE PUBLICO')]
pub = pub.sort_values( by=["CEDULA_COD", "ANIO", "MES"], ascending=[True, False, False] )

vol_ext = afi_uni[(afi_uni['SECTOR']=='69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ') |
                  (afi_uni['SECTOR']=='89-AFILIACION DOMICILIADO EN EL EXTERIOR')]
vol_ext = vol_ext.sort_values( by=["CEDULA_COD", "ANIO", "MES"], ascending=[True, False, False] )

vol_ecu = afi_uni[afi_uni['SECTOR']=='90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA']
vol_ecu = vol_ecu.sort_values( by=["CEDULA_COD", "ANIO", "MES"], ascending=[True, False, False] )

In [21]:
#Desagregación por sector
print('De ',  afi_uni['CEDULA_COD'].nunique(), ' personas que tienen un único sector se tiene:', '\n')
print( pri['CEDULA_COD'].nunique(), ' personas que están en el sector PRIVADO')
print( pub['CEDULA_COD'].nunique(), ' personas que están en el sector PUBLICO')
print( ind['CEDULA_COD'].nunique(), ' personas que están en el sector INDEPENDIENTE')
print( vol_ext['CEDULA_COD'].nunique(), ' personas que están en el sector  Voluntario residente en el exterior')
print( vol_ecu['CEDULA_COD'].nunique(), ' personas que están en el sector Voluntario residente en el ecuador')

De  391458  personas que tienen un único sector se tiene: 

177635  personas que están en el sector PRIVADO
145992  personas que están en el sector PUBLICO
46592  personas que están en el sector INDEPENDIENTE
1994  personas que están en el sector  Voluntario residente en el exterior
19245  personas que están en el sector Voluntario residente en el ecuador


In [22]:
#391458-177626-145990-32011
177635+145992+46592+1994+19245

391458

In [23]:
#Función para crear los grupos de salarios de 12 meses consecutivos para cada cedula
def asignar_grupo(data, n_grupo):
    df = data.copy()
    df['GRUPO'] = (df.groupby('CEDULA_COD').cumcount() // n_grupo ) + 1
    df['SAL_PROM_GRUPO'] = df.groupby(['CEDULA_COD', 'GRUPO'])['SALARIO'].transform('mean')
    return df

#Función para agrupar los valores de los sueldos
def valores_unicos_grupos(data):
    df = data.copy()
    df1 = df.groupby(['CEDULA_COD', 'GRUPO'])['SAL_PROM_GRUPO'].unique()
    df1 = df1.reset_index()
    df1['SAL_PROM_GRUPO'] = df1['SAL_PROM_GRUPO'].apply(lambda x: x[0])
    df1 = df1.sort_values(by = ['CEDULA_COD','SAL_PROM_GRUPO'], ascending=[True,False])
    return df1

#Función para seleccionar los 5 mejores años de sueldo promedio
def sel_top_salarios( data, top ):
    df = data.copy()
    top_sal = df.groupby('CEDULA_COD')['SAL_PROM_GRUPO'].nlargest(top).reset_index(level=0, drop=True)
    indices = top_sal.index
    df1 = df.loc[indices].reset_index(drop=True)
    return df1

#Función para seleccionar los grupos de sueldos
def sel_grupo(data1, data2):
    df1 = data1.copy()
    df2 = data2.copy()
    combinaciones = set(zip(df2['CEDULA_COD'], df2['GRUPO']))
    df1['SEL_GRUPO'] = [(cedula, grupo) in combinaciones for cedula, grupo in zip(df1['CEDULA_COD'], df1['GRUPO'])]
    df1['SEL_GRUPO'] = df1['SEL_GRUPO'].astype(int)
    return df1

#Función para insertar las fecha de inicio y fin de los cinco mejores años
def fec_sel_grupo(data):
    df = data.copy()
    grupo = data[data['SEL_GRUPO'] == 1].copy()
    grupo['ANIO_MAX'] = grupo['CEDULA_COD'].map( grupo.groupby('CEDULA_COD')['ANIO'].max() )
    grupo['MES_MAX']  = grupo['CEDULA_COD'].map(grupo[ grupo['ANIO']==grupo['ANIO_MAX']].groupby('CEDULA_COD')['MES'].max())
    grupo['FEC_INI'] = grupo['ANIO_MAX'].astype(str) + '-' + grupo['MES_MAX'].astype(str)
    grupo = grupo.groupby('CEDULA_COD')['FEC_INI'].unique().astype(str).str.strip('[ ]').str.replace("'", '')
    df['FEC_INI'] = df['CEDULA_COD'].map(grupo)
    
    grupo = data[data['SEL_GRUPO'] == 1].copy()
    grupo['ANIO_MIN'] = grupo['CEDULA_COD'].map( grupo.groupby('CEDULA_COD')['ANIO'].min() )
    grupo['MES_MIN']  = grupo['CEDULA_COD'].map(grupo[ grupo['ANIO']==grupo['ANIO_MIN']].groupby('CEDULA_COD')['MES'].min())
    grupo['FEC_FIN'] = grupo['ANIO_MIN'].astype(str) + '-' + grupo['MES_MIN'].astype(str)
    grupo = grupo.groupby('CEDULA_COD')['FEC_FIN'].unique().astype(str).str.strip('[ ]').str.replace("'", '')
    df['FEC_FIN'] = df['CEDULA_COD'].map(grupo)
    return df

#Función para calcular la base de calculo teórica
def base_cal(data):
    df = data.copy()
    grupo = data[data['SEL_GRUPO'] == 1].copy()
    df_prom = grupo.groupby('CEDULA_COD')['SALARIO'].mean()
    df['BASE_CAL'] = df['CEDULA_COD'].map( df_prom )
    return df

#Función para calcular los valores atipicos dentro de los grupos seleccionados- Método Caja y Bigote
def ati_base_cal_m1( data, inferior):
    df = data.copy()
    grupo = data[data['SEL_GRUPO'] == 1].copy()
    Q1 = grupo.groupby('CEDULA_COD')['SALARIO'].quantile(0.25)
    Q3 = grupo.groupby('CEDULA_COD')['SALARIO'].quantile(0.75)
    IQR = Q3-Q1
    LI = Q1 - 1.5 * IQR
    LS = Q3 + 1.5 * IQR
    df['IQR'] = df['CEDULA_COD'].map( IQR )
    df['LI'] = df['CEDULA_COD'].map( LI )
    df['LS'] = df['CEDULA_COD'].map( LS )
    
    if inferior==True:
        df['ATI_M1'] = ((df['SALARIO'] < df['LI']) | (df['SALARIO'] > df['LS'])).astype(int)
    else:
        df['ATI_M1'] = ( df['SALARIO'] > df['LS'] ).astype(int)
    
    grupo = df[df['SEL_GRUPO'] == 1].copy()
    n_ati = ( grupo.groupby('CEDULA_COD')['ATI_M1'].sum()/grupo.groupby('CEDULA_COD')['ATI_M1'].count() ) * 100
    df['PATI_M1'] = df['CEDULA_COD'].map( n_ati )
    df = df.drop(['IQR', 'LI', 'LS'], axis=1)
    return df

#Función para calcular la base de calculo sin los valores atipicos dentro de los grupos seleccionados método 1
def base_cal_sin_ati_m1(data):
    df = data.copy()
    grupo = df[ (df['SEL_GRUPO'] == 1 ) & (df['ATI_M1'] != 1)].copy()
    df_prom = grupo.groupby('CEDULA_COD')['SALARIO'].mean()
    df['BCSATI_M1'] = df['CEDULA_COD'].map( df_prom ) #BCSATI_M1=Base de Calculo sin atipico del Método 1
    df['INC_M1'] = df['BASE_CAL']/df['BCSATI_M1']
    return df

#Función para corregir atipicos que no sean sueldos con el SBU
SBU = pd.DataFrame( { 'ANIO':[2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
                              2021, 2022, 2023],
                      'VALOR':[105, 122, 136, 150, 160, 170, 200, 218, 240, 264, 292, 318, 340, 354, 366, 375, 386, 394, 400, 
                             400, 425, 450]})
def sbu_ajuste_m1(data, SBU):
    df = data.copy()
    aux = SBU.groupby('ANIO')['VALOR'].unique().astype(str).str.strip('[ ]').str.replace("'", '')
    df['SBU'] = df['ANIO'].map( aux ).astype(float)
    df['CO_ATI_M1'] = df['ATI_M1']
    df.loc[(df['ATI_M1'] == 1) & (df['SALARIO'] == df['SBU']), 'CO_ATI_M1'] = 0  #C0_AT1_M1 es correción de atipicos metodo 1
    
    #Corrección de atipicos
    grupo = df[ (df['SEL_GRUPO'] == 1 ) & (df['CO_ATI_M1'] != 1)].copy()
    df_prom = grupo.groupby('CEDULA_COD')['SALARIO'].mean()
    df['CO_BCSATI_M1'] = df['CEDULA_COD'].map( df_prom )
    df['CO_INC_M1'] = df['BASE_CAL']/df['CO_BCSATI_M1']
    
    grupo = df[df['SEL_GRUPO'] == 1].copy()
    n_ati = ( grupo.groupby('CEDULA_COD')['CO_ATI_M1'].sum()/grupo.groupby('CEDULA_COD')['CO_ATI_M1'].count() ) * 100
    df['CO_PATI_M1'] = df['CEDULA_COD'].map( n_ati )
    return df

In [24]:
lista_sectores = ['pri', 'ind','pub', 'vol_ext', 'vol_ecu']

n_grupo = 12  # Grupos de 12 porque corresponden a los 12 meses consecutivos
top = 5  # 5 Mejores años de sueldo
inferior = False

# Lista para almacenar los DataFrames resultantes de aplicar las funciones
resultados = pd.DataFrame()

for sector in lista_sectores:  # Lista de nombres de DataFrames
    # Obtener el DataFrame correspondiente al nombre usando globals()
    sect = globals()[sector]
    # Aplicar las funciones a cada DataFrame
    df = asignar_grupo(sect, n_grupo)
    df1 = valores_unicos_grupos(df)
    df2 = sel_top_salarios(df1, top)
    df = sel_grupo(df, df2)
    df = fec_sel_grupo(df)
    df = base_cal(df)
    df = ati_base_cal_m1(df, inferior)
    df = base_cal_sin_ati_m1(df)
    df = sbu_ajuste_m1(df, SBU)
    
    # Añadir el DataFrame resultante a la lista de resultados
    resultados = pd.concat([resultados, df], ignore_index=True)
    sect = None
    df = None
    df1 = None
    df2 = None

In [25]:
resultados

,CEDULA_COD,ANIO,MES,SALARIO,PORCENTAJE_AP,SECTOR,GRUPO,SAL_PROM_GRUPO,SEL_GRUPO,FEC_INI,...,BASE_CAL,ATI_M1,PATI_M1,BCSATI_M1,INC_M1,SBU,CO_ATI_M1,CO_BCSATI_M1,CO_INC_M1,CO_PATI_M1
0,115,2006,4,11.67,20.5,PRIVADO,1,312.407778,1,2006-4,...,312.407778,0,0.0,312.407778,1.0,160.0,0,312.407778,1.0,0.0
1,115,2006,3,350.00,20.5,PRIVADO,1,312.407778,1,2006-4,...,312.407778,0,0.0,312.407778,1.0,160.0,0,312.407778,1.0,0.0
2,115,2006,2,350.00,20.5,PRIVADO,1,312.407778,1,2006-4,...,312.407778,0,0.0,312.407778,1.0,160.0,0,312.407778,1.0,0.0
3,115,2006,1,350.00,20.5,PRIVADO,1,312.407778,1,2006-4,...,312.407778,0,0.0,312.407778,1.0,160.0,0,312.407778,1.0,0.0
4,115,2005,12,350.00,20.5,PRIVADO,1,312.407778,1,2006-4,...,312.407778,0,0.0,312.407778,1.0,150.0,0,312.407778,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43322365,20847292,2016,3,366.00,17.5,90-AFILIACION DOMICILIADO EN EL PAIS / SIN REL...,1,362.000000,1,2016-4,...,362.000000,0,0.0,362.000000,1.0,366.0,0,362.000000,1.0,0.0
43322366,20847292,2016,2,366.00,20.5,90-AFILIACION DOMICILIADO EN EL PAIS / SIN REL...,1,362.000000,1,2016-4,...,362.000000,0,0.0,362.000000,1.0,366.0,0,362.000000,1.0,0.0
43322367,20847292,2016,1,366.00,20.5,90-AFILIACION DOMICILIADO EN EL PAIS / SIN REL...,1,362.000000,1,2016-4,...,362.000000,0,0.0,362.000000,1.0,366.0,0,362.000000,1.0,0.0
43322368,20847292,2015,12,354.00,20.5,90-AFILIACION DOMICILIADO EN EL PAIS / SIN REL...,1,362.000000,1,2016-4,...,362.000000,0,0.0,362.000000,1.0,354.0,0,362.000000,1.0,0.0


In [26]:
resultados['CEDULA_COD'].nunique()

391458

In [117]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [252]:
data = resultados[resultados['SECTOR']=='PRIVADO'].copy()
data

,CEDULA_COD,ANIO,MES,SALARIO,PORCENTAJE_AP,SECTOR,GRUPO,SAL_PROM_GRUPO,SEL_GRUPO,FEC_INI,...,BASE_CAL,ATI_M1,PATI_M1,BCSATI_M1,INC_M1,SBU,CO_ATI_M1,CO_BCSATI_M1,CO_INC_M1,CO_PATI_M1
0,115,2006,4,11.67,20.5,PRIVADO,1,312.407778,1,2006-4,...,312.407778,0,0.0,312.407778,1.0,160.0,0,312.407778,1.0,0.0
1,115,2006,3,350.00,20.5,PRIVADO,1,312.407778,1,2006-4,...,312.407778,0,0.0,312.407778,1.0,160.0,0,312.407778,1.0,0.0
2,115,2006,2,350.00,20.5,PRIVADO,1,312.407778,1,2006-4,...,312.407778,0,0.0,312.407778,1.0,160.0,0,312.407778,1.0,0.0
3,115,2006,1,350.00,20.5,PRIVADO,1,312.407778,1,2006-4,...,312.407778,0,0.0,312.407778,1.0,160.0,0,312.407778,1.0,0.0
4,115,2005,12,350.00,20.5,PRIVADO,1,312.407778,1,2006-4,...,312.407778,0,0.0,312.407778,1.0,150.0,0,312.407778,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19759014,23652869,1990,8,56.65,20.5,PRIVADO,5,56.650000,1,1995-2,...,56.650000,0,0.0,56.650000,1.0,NaN,0,56.650000,1.0,0.0
19759015,23652869,1990,7,56.65,20.5,PRIVADO,5,56.650000,1,1995-2,...,56.650000,0,0.0,56.650000,1.0,NaN,0,56.650000,1.0,0.0
19759016,23652869,1990,6,56.65,20.5,PRIVADO,5,56.650000,1,1995-2,...,56.650000,0,0.0,56.650000,1.0,NaN,0,56.650000,1.0,0.0
19759017,23652869,1990,5,56.65,20.5,PRIVADO,5,56.650000,1,1995-2,...,56.650000,0,0.0,56.650000,1.0,NaN,0,56.650000,1.0,0.0


In [263]:
df=data.copy()
grupo = df[ (df['SEL_GRUPO'] == 1 )].copy()
cel=grupo['CEDULA_COD'].unique()

In [264]:
for cedula in cel:
    aux=None
    aux=pd.DataFrame( {'t': range(1, len(grupo[grupo['CEDULA_COD']==cedula]) + 1), 
                   'n': grupo[grupo['CEDULA_COD']==cedula]['SALARIO'].to_numpy() } ).values
    
    try:
        Z = linkage(aux, method='single', metric='euclidean')
        
    except ValueError:
        aux1 = pd.DataFrame()
        aux1['CLUSTER'] = - 1
        aux1['UMBRAL'] = -1
        aux1['CEDULA_COD'] = cedula
        aux1_um = aux1.groupby('CEDULA_COD')['UMBRAL'].unique().astype(str).str.strip('[ ]').str.replace("'", '').astype(float)
        data['UMBRAL'] = data['CEDULA_COD'].map( aux1_um )
        aux1.index = grupo[grupo['CEDULA_COD']==cedula].index
        data.loc[:, 'CLUSTER'] = data.index.map(aux1['CLUSTER'].to_dict())
        continue
    else:
    
        dendrograma = dendrogram(Z, no_plot=True)
        umbral = encontrar_umbral(dendrograma, aux)  
        clusters = fcluster(Z, umbral, criterion='distance')
        aux1 = pd.DataFrame()
        aux1['CLUSTER'] = clusters
        aux1['UMBRAL'] = umbral
        aux1['CEDULA_COD'] = cedula
        aux1_um = aux1.groupby('CEDULA_COD')['UMBRAL'].unique().astype(str).str.strip('[ ]').str.replace("'", '').astype(float)
    
        data['UMBRAL'] = data['CEDULA_COD'].map( aux1_um )
    
        aux1.index = grupo[grupo['CEDULA_COD']==cedula].index
        data.loc[:, 'CLUSTER'] = data.index.map(aux1['CLUSTER'].to_dict())

ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

In [272]:
cedula

265

In [240]:
data[(data['SEL_GRUPO']==1) & (data['CEDULA_COD']==265)]

,CEDULA_COD,ANIO,MES,SALARIO,PORCENTAJE_AP,SECTOR,GRUPO,SAL_PROM_GRUPO,SEL_GRUPO,FEC_INI,...,PATI_M1,BCSATI_M1,INC_M1,SBU,CO_ATI_M1,CO_BCSATI_M1,CO_INC_M1,CO_PATI_M1,UMBRAL,CLUSTER
327,265,1998,12,11.0,20.5,PRIVADO,1,11.0,1,1998-12,...,0.0,11.0,1.0,NaN,0,11.0,1.0,0.0,NaN,NaN


In [244]:
data1=data[(data['CEDULA_COD']==265) & (data['SEL_GRUPO']==1)]

a={'t': range(1, len(data1) + 1),
   'n': data1['SALARIO'].to_numpy()}

a=pd.DataFrame(a).values

# Definir el umbral automáticamente
def encontrar_umbral(dendrograma, distancia):
    deltas = []
    for i in range(1, len(dendrograma['icoord'])):
        delta = abs(dendrograma['icoord'][i][2] - dendrograma['icoord'][i-1][2])
        deltas.append(delta)
    max_delta_index = deltas.index(max(deltas))
    return (dendrograma['dcoord'][max_delta_index][1] + dendrograma['dcoord'][max_delta_index+1][1]) / 2
a

array([[ 1., 11.]])

In [246]:
Z = linkage(a, method='single', metric='euclidean')
Z

ValueError: The number of observations cannot be determined on an empty distance matrix.

In [242]:
Z = linkage(a, method='single', metric='euclidean')
dendrograma = dendrogram(Z, no_plot=True)
umbral_automatico = encontrar_umbral(dendrograma, a)

print("Umbral automático:", umbral_automatico)

# Obtener los clusters usando fcluster con el umbral automático
clusters = fcluster(Z, umbral_automatico, criterion='distance')

# Crear un DataFrame con los datos y los clusters asignados
df = pd.DataFrame(a, columns=['Índice', 'Salario'])
df['Cluster'] = clusters
df
# Identificar los valores atípicos
#valores_atipicos = df[df['Cluster'] != min(df['Cluster'])]
#valores_atipicos

ValueError: The number of observations cannot be determined on an empty distance matrix.

In [243]:
dendograma

{'icoord': [[15.0, 15.0, 25.0, 25.0],
  [65.0, 65.0, 75.0, 75.0],
  [55.0, 55.0, 70.0, 70.0],
  [45.0, 45.0, 62.5, 62.5],
  [95.0, 95.0, 105.0, 105.0],
  [115.0, 115.0, 125.0, 125.0],
  [100.0, 100.0, 120.0, 120.0],
  [85.0, 85.0, 110.0, 110.0],
  [145.0, 145.0, 155.0, 155.0],
  [175.0, 175.0, 185.0, 185.0],
  [165.0, 165.0, 180.0, 180.0],
  [150.0, 150.0, 172.5, 172.5],
  [215.0, 215.0, 225.0, 225.0],
  [205.0, 205.0, 220.0, 220.0],
  [195.0, 195.0, 212.5, 212.5],
  [161.25, 161.25, 203.75, 203.75],
  [135.0, 135.0, 182.5, 182.5],
  [97.5, 97.5, 158.75, 158.75],
  [53.75, 53.75, 128.125, 128.125],
  [265.0, 265.0, 275.0, 275.0],
  [255.0, 255.0, 270.0, 270.0],
  [295.0, 295.0, 305.0, 305.0],
  [285.0, 285.0, 300.0, 300.0],
  [262.5, 262.5, 292.5, 292.5],
  [245.0, 245.0, 277.5, 277.5],
  [235.0, 235.0, 261.25, 261.25],
  [90.9375, 90.9375, 248.125, 248.125],
  [35.0, 35.0, 169.53125, 169.53125],
  [315.0, 315.0, 325.0, 325.0],
  [335.0, 335.0, 345.0, 345.0],
  [355.0, 355.0, 365.0, 36

In [ ]:
#Aplicación de funciones por parte
n_grupo = 12 # Grupos de 12 porque corresponde a los 12 meses consecutivos
df = asignar_grupo( pri, n_grupo )
df1 = valores_unicos_grupos( df )
top = 5 # 5 Mejores años de sueldo
df2 = sel_top_salarios( df1, top )
df = sel_grupo(df, df2 )
df = fec_sel_grupo( df )
df = base_cal(df)
inferior=False
df = ati_base_cal_m1( df, inferior)
df = base_cal_sin_ati_m1( df )
df = sbu_ajuste_m1(df, SBU)


In [ ]:
#Verificación de la utilidad de las funciones
#asignar_grupo
# aux=df[df['CEDULA_COD']==100018].pivot_table(index=('ANIO'), columns=('MES'), values=('SALARIO'),aggfunc=lambda x: x).sort_index(ascending=False)
# ordered_columns = aux.columns.sort_values(ascending=False)
# aux = aux[ordered_columns]
# aux  #--promedio del grupoo 2 OK (436.5533333333333)
#------------------------------------------------------------------------------------------------------
#valores_unicos_grupos
# df1[df1['CEDULA_COD']==100018] si estan ordenados desde el mayor al menor
#------------------------------------------------------------------------------------------------------
#sel_top_salarios
# df2[df2['CEDULA_COD']==100018] selecciona los 5 grupos de mejor sueldo
#------------------------------------------------------------------------------------------------------
#sel_grupo
# aux=df[df['CEDULA_COD']==271].pivot_table(index=('ANIO'), columns=('MES'), values=('SALARIO'),aggfunc=lambda x: x).sort_index(ascending=False)
# ordered_columns = aux.columns.sort_values(ascending=False)
# aux = aux[ordered_columns]
# aux
# df1[df1['CEDULA_COD']==271] 
# df2[df2['CEDULA_COD']==271]
# df[df['CEDULA_COD']== 271].pivot_table(index=('GRUPO'), values=('SEL_GRUPO'))
# df[ (df['CEDULA_COD']==271) & (df['SEL_GRUPO']==1)].head(60)  #--Se verifica que si selecciona los grupos correctos
#-------------------------------------------------------------------------------------------------------
# fec_sel_grupo
# df[ (df['CEDULA_COD']==100018) & (df['SEL_GRUPO']==1)] #--Se verifica que funciona bien
#-------------------------------------------------------------------------------------------------------
# base_cal
# df[(df['CEDULA_COD']==271) & (df['SEL_GRUPO']==1)]['SALARIO'].mean()  #406.14783333333315 / 4362.616666666671
# df[(df['CEDULA_COD']==271) & (df['SEL_GRUPO']==1)]  #--Se verifica que si saca el promedio de los mejores sueldos
#-------------------------------------------------------------------------------------------------------
# ati_base_cal_m1
# df5[(df5['CEDULA_COD']==271) & (df['SEL_GRUPO']==1)]['SALARIO'].quantile(0.25) # 4275.95
# df5[(df5['CEDULA_COD']==271) & (df['SEL_GRUPO']==1)]['SALARIO'].quantile(0.75) # 4275.95
# IQR = 4275.95-4275.95 = 0 # SE verifica que si sale 0
# LI = Q1 - 1.5 * IQR / 0
# LS = Q3 + 1.5 * IQR / 0
# df5[(df5['CEDULA_COD']==100018) & (df['SEL_GRUPO']==1)]['SALARIO'].quantile(0.25) # 381.0
# df5[(df5['CEDULA_COD']==100018) & (df['SEL_GRUPO']==1)]['SALARIO'].quantile(0.75) # 427.3475
# IQR = 427.3475-381.0 # SE verifica que si sale 46.347500000000025
# LI = Q1 - 1.5 * IQR / 311.47875
# LS = Q3 + 1.5 * IQR / 450.52125  #Se verifica que los valores son atipicos
# ----------------------------------------------------------------------------------------------------------
# base_cal_sin_ati_m1
# df[ df['INC_M1']>1.50]
# df[df['CEDULA_COD']==1883].head(60) #Se verifica que si cambia
#-----------------------------------------------------------------------------------------------------------
# sbu_ajuste_m1
#df[(df['SEL_GRUPO']==1) & (df['ATI_M1']==1) & (df['CO_ATI_M1']==0)]
#df[(df['SEL_GRUPO']==1) & (df['CEDULA_COD']==1248)][['SALARIO', 'BASE_CAL', 'ATI_M1','PATI_M1','BCSATI_M1','INC_M1','SBU','CO_ATI_M1','CO_BCSATI_M1','CO_INC_M1','CO_PATI_M1']]
# Se verifica que se corrigen en función del SBU

In [ ]:
list_sector = ['pri', 'pub' ]
globals()list_sector[0]

In [ ]:
for df_name in ['pri', 'pub']:  # Lista de nombres de DataFrames
    # Obtener el DataFrame correspondiente al nombre usando globals()
    df =  globals()[df_name] 
    

In [ ]:
#Sector privado
inicio = time.time()
pri_t = pri.pivot_table(index=('CEDULA_COD'), columns=('ANIO', 'MES'), values=('SALARIO'),aggfunc=lambda x: x)
pri_t.sort_values(by='CEDULA_COD',inplace=True)
fin = time.time()
print('Tiempo de ejecución:\n')
print('horas', (fin-inicio)//3600 )
print('minutos', ((fin-inicio)%3600)//60 )
print('segundos', (fin-inicio)%60 )

In [ ]:
pri_t

In [ ]:
pri_t['Bas_Cal'] = pd.NA
pri_t['Fec_Ini'] = pd.NA
pri_t['Fec_Fin'] = pd.NA

In [ ]:
inicio = time.time()

meses = 12
base = 5

for persona in pri_t.index[:10000]:
    
    col_sel=pri_t.loc[persona].notna()
    columnas_sin_nan = pri_t.columns[col_sel & ( pri_t.columns != "CEDULA_COD")  & ( pri_t.columns != "Bas_Cal") & ( pri_t.columns != "Fec_Ini") & ( pri_t.columns != "Fec_Fin")  ][::-1]
    grupos = [columnas_sin_nan[i:i+meses] for i in range(0, len(columnas_sin_nan), meses)]
    
    sumas_por_grupo = {}
    for grupo in grupos:
        suma_grupo = pri_t[grupo].mean(axis=1).values[0]
        sumas_por_grupo[tuple(grupo)] = suma_grupo
        # Ordenar los grupos por su suma en orden descendente
        grupos_ordenados = sorted(sumas_por_grupo.items(), key=lambda x: x[1], reverse=True)[:base]
    
    componentes1 = [[par for par in tupla[0]] for tupla in grupos_ordenados]
    componentes = [item for sublist in componentes1 for item in sublist]
    
    clave = lambda tupla: tupla[0] * 100 + tupla[1]
    # Encontrar el par de año y mes mínimo y máximo usando la clave personalizada
    par_minimo = min((min(sublista, key=clave ) for sublista in componentes1), key=clave )
    par_maximo = max((max(sublista, key=clave ) for sublista in componentes1), key=clave )
    
    pri_t.loc[pri_t.index==persona,'Bas_Cal'] = pri_t.iloc[pri_t.index==persona][componentes].mean(axis=1)
    pri_t.loc[pri_t.index==persona,'Fec_Ini'] = "{}/{}".format(par_maximo[0], str(par_maximo[1]).zfill(2))
    pri_t.loc[pri_t.index==persona,'Fec_Fin'] = "{}/{}".format(par_minimo[0], str(par_minimo[1]).zfill(2))
        
fin = time.time()
print('Tiempo de ejecución:\n')
print('horas', (fin-inicio)//3600 )
print('minutos', ((fin-inicio)%3600)//60 )
print('segundos', (fin-inicio)%60 )   

In [ ]:
pri_t

In [ ]:
col_sel=pri.loc[persona].notna()
columnas_sin_nan = pri.columns[col_sel][::-1]
columnas_sin_nan
# grupos = [columnas_sin_nan[i:i+meses] for i in range(0, len(columnas_sin_nan), meses)]
    
# sumas_por_grupo = {}
# for grupo in grupos:
#     suma_grupo = pri[grupo].mean(axis=1).values[0]
#     sumas_por_grupo[tuple(grupo)] = suma_grupo
#         # Ordenar los grupos por su suma en orden descendente
#     grupos_ordenados = sorted(sumas_por_grupo.items(), key=lambda x: x[1], reverse=True)[:base]
    
# componentes1 = [[par for par in tupla[0]] for tupla in grupos_ordenados]
# componentes = [item for sublist in componentes1 for item in sublist]
    
# clave = lambda tupla: tupla[0] * 100 + tupla[1]
#     # Encontrar el par de año y mes mínimo y máximo usando la clave personalizada
# par_minimo = min((min(sublista, key=clave ) for sublista in componentes1), key=clave )
# par_maximo = max((max(sublista, key=clave ) for sublista in componentes1), key=clave )
    
# pri.loc[pri.index==persona,'Base_calculada'] = pri.iloc[pri.index==persona][componentes].mean(axis=1)
# pri.loc[pri.index==persona,'Fec_Ini'] = pd.to_datetime('{},{}'.format( par_maximo[0], par_maximo[1]), format='%Y, %m')
# pri.loc[pri.index==persona,'Fec_Fin'] = pd.to_datetime('{},{}'.format( par_minimo[0], par_minimo[1]), format='%Y, %m')

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql.functions import concat_ws
from pyspark.sql.types import IntegerType

spark = SparkSession.builder \
    .appName("YourApp") \
    .getOrCreate()

# Definir meses y base
meses = 12
base = 5



In [ ]:
pri_spark = spark.createDataFrame(pd.DataFrame(pri))
pri_spark

In [ ]:
#Para determinar las columnas con valores diferentes a NaN
col_sel=pri.loc['100018'].notna()
columnas_sin_nan = pri.columns[col_sel][::-1]

In [ ]:
meses = 12
grupos = [columnas_sin_nan[i:i+meses] for i in range(0, len(columnas_sin_nan), meses)]

In [ ]:
pri.iloc[pri.index=='100018'].stack(0)

In [ ]:
440.15+415.21+512.47+399.00+399.00+403.99+399.00+463.84+416.46+266.00+541.29+419.10

In [ ]:
base = 5
sumas_por_grupo = {}
for grupo in grupos:
    suma_grupo = pri[grupo].mean(axis=1).values[0]
    sumas_por_grupo[tuple(grupo)] = suma_grupo

# Ordenar los grupos por su suma en orden descendente
grupos_ordenados = sorted(sumas_por_grupo.items(), key=lambda x: x[1], reverse=True)[:base]
#grupos_ordenados

In [ ]:
componentes1 = [[par for par in tupla[0]] for tupla in grupos_ordenados]
componentes = [item for sublist in componentes1 for item in sublist]

clave_personalizada = lambda tupla: tupla[0] * 100 + tupla[1]

# Encontrar el par de año y mes mínimo y máximo usando la clave personalizada
par_minimo = min((min(sublista, key=clave_personalizada) for sublista in componentes1), key=clave_personalizada)
par_maximo = max((max(sublista, key=clave_personalizada) for sublista in componentes1), key=clave_personalizada)

In [ ]:
pri['Base_calculada']=pd.NA
pri['Fec_Ini'] = pd.NA
pri['Fec_Fin'] = pd.NA

In [ ]:
pri['Base_calculada'] = pri.iloc[pri.index=='100018'][componentes].mean(axis=1)
pri.loc[pri.index=='100018','Fec_Ini'] = pd.to_datetime('{}, {}'.format( par_maximo[0], par_maximo[1]), format='%Y, %m')
pri.loc[pri.index=='100018','Fec_Fin'] = pd.to_datetime('{}, {}'.format( par_minimo[0], par_minimo[1]), format='%Y, %m')

In [ ]:
pri

In [ ]:
vej[vej['CEDULA_COD']==114]

In [ ]:
afi[afi['CEDULA_COD']==114]

In [ ]:
afi.loc[['114']]

In [ ]:
import pickle

# Nombre del archivo donde se guardará el objeto
nombre_archivo = 'proceso.pkl'

# Guardar el objeto en el archivo
with open(nombre_archivo, 'wb') as archivo:
    pickle.dump(pri, archivo)

In [ ]:
import pickle

# Nombre del archivo donde se guardó el objeto
nombre_archivo = 'proceso.pkl'

# Cargar el objeto desde el archivo
with open(nombre_archivo, 'rb') as archivo:
    mi_objeto_cargado = pickle.load(archivo)

print(mi_objeto_cargado)